# Binary Search

Binary Search algorithms operate on *sorted* arrays. By *sorted* we mean that there is a condition ` condition(x: T) -> bool` such that all the elements that verify the condition are on one side of the array, and all the other elements are on the other side.

There are many ways to implement binary search, so the best way to avoid out-of-bound indices or infinite loops is to stick to the same template.

```python
def binary_search(sorted_array: list[T]) -> int:
    """Returns the min index `i` for which `condition(i)` holds."""

    def condition(i: int) -> bool:
        ...

    l, r = min_inclusive_index_bound, max_inclusive_index_bound
    while l < r:
        m = (l + r) // 2
        if condition(m):
            r = m
        else:
            l = m + 1
    return l
```

Adjust the template by:
* Defining the appropriate inclusice bounds for the index
* Defining the right `condition()` function
* Returing either `l` (minimal index with `condition` holding) or `l - 1` (maximal index not verifying `condition`)

## Why it works

* Finite loop: the loop necessarily ends because the quantity `l - r` decreases by at least 1 at each step:
    * int division `//` rounds down in Python, and `l < r`, so `m < r`: the assignment `r = m` decreases `r` by at least 1
    * `l` can be `m` but the statement `l = m + 1` ensures `l` increases by at least 1
* Go left or right:
    * When the condition is false at index `m`, we know for sure m is not the minimal index for which the condition holds: we can safely increment `l` to the right of `m`
    * When the condition is true at index `m`, m could be the minimal index for which the condition holds: reassign `r` to `m`
* Return value. `l` and `r` are guaranteed to be equal at the end of the loop, so we can return either one of them.

# Bisect

The direct application of the above template is `bisect()`.

Bisect left (resp. right): returns the leftmost (resp. rightmost) index of matching value(s), or the index where the target value should be *inserted*.  
  
Bisect left returns $min \{ i, array[i] \geq target \}$  
Bisect right returns $min \{ i, array[i] > target \}$. Just turn the strict inequality below into a large one.

[Alternative implementation](https://gist.github.com/py-in-the-sky/e84fd9fc4db0da3f351631ba04b2d91b)

In [1]:
def bisect_left(array: list[int], target: int) -> int:
    left, right = 0, len(array)
    while left < right: # Strict inequality because `left == right` in the last step is guaranteed
        mid = (left + right) // 2
        if array[mid] < target:
            left = mid + 1 # `mid` cannot be the correct index, shift `left`` on its right
        else:
            right = mid # `array[mid] <= left`, so it could be to correct index
    return left # `right` works too

def bisect_right(array: list[int], target: int) -> int:
    left, right = 0, len(array)
    while left < right:
        mid = (left + right) // 2
        if array[mid] <= target:
            left = mid + 1
        else:
            right = mid
    return left

### No target

If the target is not in the array, `bisect_left()` and `bisect_right()` return the same index: the position where `target` should be inserted to maintain the sorted property.

Note: the returned value lies in the range `[0, len(array)]`.

In [2]:
print(f"{bisect_right([3], 7) = }")
print(f"{bisect_right([3, 8], 7) = }")
print(f"{bisect_right([8], 7) = }")
print(f"{bisect_left([3], 7) = }")
print(f"{bisect_left([3, 8], 7) = }")
print(f"{bisect_left([8], 7) = }")

bisect_right([3], 7) = 1
bisect_right([3, 8], 7) = 1
bisect_right([8], 7) = 0
bisect_left([3], 7) = 1
bisect_left([3, 8], 7) = 1
bisect_left([8], 7) = 0


### 1 or more occurences of the target

Array: [1, 3, 7, 7, 8], target: 7

In [3]:
print(f" Max index i with array[i] < target: bisect_left - 1. bisect_left([1, 3, 7, 7, 8], 7) - 1 = {bisect_left([1, 3, 7, 7, 8], 7) - 1}")
print(f" Min index i with array[i] <= target: bisect_left. bisect_left([1, 3, 7, 7, 8], 7) = {bisect_left([1, 3, 7, 7, 8], 7)}")
print(f" Max index i with array[i] >= target: bisect_right - 1. bisect_right([1, 3, 7, 7, 8], 7) - 1 = {bisect_right([1, 3, 7, 7, 8], 7) - 1}")
print(f" Max index i with array[i] > target: bisect_right. bisect_right([1, 3, 7, 7, 8], 7) = {bisect_right([1, 3, 7, 7, 8], 7)}")

 Max index i with array[i] < target: bisect_left - 1. bisect_left([1, 3, 7, 7, 8], 7) - 1 = 1
 Min index i with array[i] >= target: bisect_left. bisect_left([1, 3, 7, 7, 8], 7) = 2
 Max index i with array[i] >= target: bisect_right - 1. bisect_right([1, 3, 7, 7, 8], 7) - 1 = 3
 Max index i with array[i] > target: bisect_right. bisect_right([1, 3, 7, 7, 8], 7) = 4


# Applications

In [4]:
def search(nums: list[int], target: int) -> int:
    """Return -1 if `target` is not in the sorted array `nums`, else the index of `target`."""
    l, r = 0, len(nums) - 1 # The search space is different for this application
    while l < r:
        m = (l + r) // 2
        if nums[m] < target:
            l = m + 1
        else:
            r = m
    return l if nums[l] == target else -1